In [1]:
from PIL import Image
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
from joblib import dump

In [4]:
# <
createcsv(96881,108466-96881,"L")
getLetter("L.csv", "L", "L")

# >
createcsv(108467,120802-108467,"M")
getLetter("M.csv", "M", "M")

# <=
createcsv(120803,139812-120803,"N")
getLetter("N.csv", "N", "N")

# >=
createcsv(222791,234356-222791,"R")
getLetter("R.csv", "R", "R")

In [3]:
def createcsv(sk,nr,lettre):
    pd.read_csv("/home/kanto/ETUDE/Mr_Tsinjo/analyse_de_donne/11_tp_equation/data/A_Z Handwritten Data.csv",header=None,skiprows=sk,nrows=nr).to_csv(lettre+".csv",index=False,header=None)
createcsv(74447,81664-74447,"H")   
def invert(x):
    return 255-x
def getLetter(path,folder,letter):
    data=pd.read_csv(path,header=None)
    data=data.drop(data.columns[0],axis=1)
    data=data.applymap(invert)
    
    result = data.values
    for row in range(result.shape[0]):
        image = result[row].reshape((28, 28)).astype(np.uint8)
        result_image = Image.fromarray(image)
        result_image.save(folder + "/" + letter + "_" + str(row+1) + ".png"

In [12]:
createcsv(56082, 67521-56082, "E")
getLetter("E.csv", "E", "E")

#Suppresion des lignes blanches de l'image ett Attribution de l'etiquette
def supprimer_lignes_blanches(image_path,etiquette):
    # Charger l'image
    image = Image.open(image_path)
    largeur, hauteur = image.size

    # Convertir l'image en niveau de gris
    image = image.convert("L")
    
    # Convertir l'image en tableau NumPy
    image_array = np.array(image)

    # Supprimer les lignes entièrement blanches horizontales
    indices_lignes_blanches = np.all(image_array == 255, axis=1)
    image_array = image_array[~indices_lignes_blanches]

    # Supprimer les lignes entièrement blanches verticales
    indices_colonnes_blanches = np.all(image_array == 255, axis=0)
    image_array = image_array[:, ~indices_colonnes_blanches]

    # Créer une nouvelle image à partir du tableau NumPy modifié
    result_image = Image.fromarray(image_array).resize((28, 28))

    # Récupérer les pixels de l'image sans lignes blanches
    pixels = list(result_image.getdata())
    
    pixels.append(etiquette)
    return pixels

# Prend tous le images
def getAllImage(result_array, path, folder, etiquette):
    files = os.listdir(path + "/" + folder)
    for file in files:
        result_array.append(supprimer_lignes_blanches(path + "/" + folder + "/" + file, etiquette))

def prepareAllData():
    # Tableau contenant tous les données avec leur étiquette
    result_array = []
    
    # Path contenant les données
    data_source = "/home/kanto/ETUDE/Mr_Tsinjo/analyse_de_donne/11_tp_equation/data/CompleteImages/All data (Compressed)/"
    # etiquettage de tous les images
    #  10 -> *   11 -> +  12 -> -  13 -> %
    getAllImage(result_array, data_source, '0', 0)
    getAllImage(result_array, data_source, '1', 1)
    getAllImage(result_array, data_source, '2', 2)
    getAllImage(result_array, data_source, '3', 3)
    getAllImage(result_array, data_source, '4', 4)
    getAllImage(result_array, data_source, '5', 5)
    getAllImage(result_array, data_source, '6', 6)
    getAllImage(result_array, data_source, '7', 7)
    getAllImage(result_array, data_source, '8', 8)
    getAllImage(result_array, data_source, '9', 9)
    getAllImage(result_array, data_source, '*', 10)    
    getAllImage(result_array, data_source, '+', 11)    
    getAllImage(result_array, data_source, '-', 12)        
    getAllImage(result_array, data_source, '%', 13)  
    getAllImage(result_array, data_source, '[', 14)
    getAllImage(result_array, data_source, ']', 15)
    getAllImage(result_array, "./", "X", 16)
    getAllImage(result_array, "./", "E", 17)
    getAllImage(result_array, "./", "V", 18)
    getAllImage(result_array, "./", "L", 19) # <
    getAllImage(result_array, "./", "M", 20) # >
    getAllImage(result_array, "./", "N", 21) # <=
    getAllImage(result_array, "./", "R", 22) # >=
    
    return np.array(result_array)
image_array=prepareAllData()

#print(image_array[:,-1])

In [13]:
# Attributs et etiquettes
X = image_array[:, 0:-1]
y = image_array[:, -1]

# division donnees de test et entrainement
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 0)

In [14]:
# Entrainement
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(random_state=0)
clf = clf.fit(X_train, y_train)

In [15]:
y_pred = clf.predict(X_test)

In [8]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[4056    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0]
 [   1 3776    2    0    2    1    0   54    1    8    0    1  148   78
     5    9]
 [   0   10 4102    1    2    1    1   10    4    4    0    0    2    1
    13    4]
 [   0    1    1 4286    0   11    0    3   10   13    0    0    0    0
     0    4]
 [   0   11    2    0 4276    0    2    4    0   12   10   12    3    2
     1    0]
 [   0    0    0   10    3 4124    3    4    0    3    1    0    0    0
     0    0]
 [   2    0    0    0    0    0 4087    0    4    2    0    0    0    0
    11    0]
 [   0    7    6    2   12    1    0 3928    2   11    1   17   11   26
     1   18]
 [   0    1    1    2    0    0    2    0 4032   11    0    0    0    0
     0    0]
 [  11    6    4   17    4    2    0   11    7 4050    0    0    5    2
     0   14]
 [   0    1    0    0    7    0    0    0    0    0 4026   17    0    0
     0    0]
 [   0   34    1    1   14    0    1   10    0    1   21 4321   3

In [16]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.981142699547241

In [20]:
def supprimer(image_path):
    # Charger l'image
    image = Image.open(image_path)
    largeur, hauteur = image.size

    # Convertir l'image en niveau de gris
    image = image.convert("L")
    
    # Convertir l'image en tableau NumPy
    image_array = np.array(image)

    # Supprimer les lignes entièrement blanches horizontales
    indices_lignes_blanches = np.all(image_array == 255, axis=1)
    image_array = image_array[~indices_lignes_blanches]

    # Supprimer les lignes entièrement blanches verticales
    indices_colonnes_blanches = np.all(image_array == 255, axis=0)
    image_array = image_array[:, ~indices_colonnes_blanches]

    # Créer une nouvelle image à partir du tableau NumPy modifié
    result_image = Image.fromarray(image_array)
    
     # Redimensionner l'image à 28x28
    result_image = result_image.resize((28, 28))
    
     # Enregistrer l'image modifiée
    result_image.save("image_sans_lignes_blanches.png")

    # Charger à nouveau l'image sans lignes blanches
    image_sans_lignes_blanches = Image.open("image_sans_lignes_blanches.png")
    # Récupérer les pixels de l'image sans lignes blanches
    pixels = list(result_image.getdata())
    
   
    return pixels
pixels=supprimer("/home/kanto/ETUDE/Mr_Tsinjo/analyse_de_donne/V/V_1.png")
pixels = np.array([pixels])
#pixels=improveData(pixels)
print(clf.predict(pixels))

NameError: name 'clf' is not defined

In [11]:
dump(clf, "model_x_v_e.joblib")

['model_h.joblib']